In [1]:
import string
import pandas as pd
import numpy as np
import re
import string

In [2]:
data = pd.read_csv('prabowo_mkdataset.csv', encoding='latin1')
data.head(10)

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2023-11-26T22:41:45Z,yuliyan susilo,"Coba pikir,,,ini jelas salah,,tp sedikit orang...",0
1,2023-11-26T21:08:50Z,Dimin Suherman,"Udh mending pilih anis, udh jelas jelas kan, ...",0
2,2023-11-21T15:33:57Z,P Mamayan,mba nana harusnnya jangan framing kubu yg nga ...,0
3,2023-11-19T16:23:30Z,VIRAL LUCU,Saya FANATIK Jokowi tapi stelah terjadi REKAYA...,0
4,2023-11-18T14:43:31Z,Novita Achmad,"assalamualaikum mbak,,,saya koq gak lihat meng...",0
5,2023-11-17T04:18:19Z,Munirul Khakim,Apakah ini pelanggaran moral etik berat dinega...,0
6,2023-11-16T08:12:46Z,BOSSJOE tv,Fakta: satu satunya orang di indonesia yg meng...,0
7,2023-11-15T10:44:51Z,didin saripudin,Setali tiga uang paman sama ponakan sama2 gak ...,0
8,2023-11-14T12:45:01Z,AKMAL MAXIMAL,"Jangan sampai mereka berkuasa, kita anak indon...",0
9,2023-11-14T08:24:22Z,Alfa Beta,Lawak negeri ini kepala dinas pendidikan kampa...,0


In [3]:
#cek nilai duplicate dan nilai kosong
data = data[['textDisplay']]
print(data.duplicated().sum())
print(data.isnull().sum())

4
textDisplay    0
dtype: int64


In [4]:
#menghilangkan nilai duplicate dan nilai kosong
data = data.drop_duplicates()
data= data.dropna()
print(data.duplicated().sum())
print(data.isnull().sum())

0
textDisplay    0
dtype: int64


In [5]:
def clean_twitter_text(text):
    # Menghapus mentions, hashtags, dan URL
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r',', ' ', text)
    text = re.sub(r'<br>', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

#clean dari hastag dll
data['textDisplay'] = data['textDisplay'].apply(clean_twitter_text)
#mengecilkan semua huruf
data['textDisplay'] = data['textDisplay'].str.lower()
data

,textDisplay
0,coba pikir ini jelas salah tp sedikit orang yg...
1,udh mending pilih anis udh jelas jelas kan ing...
2,mba nana harusnnya jangan framing kubu yg nga ...
3,saya fanatik jokowi tapi stelah terjadi rekaya...
4,assalamualaikum mbak saya koq gak lihat mengge...
...,...
891,kode keras dari najwa kalau gibran sampai maju...
892,saldi isra itu hakim pilihan presiden mereka s...
893,iya dong mubazir klo gk dipake
895,s7


In [6]:
def replace_words_in_sentence(dataframe, column_name, replacements):
    # Duplikat DataFrame agar tidak merubah DataFrame asli
    df_copy = dataframe.copy()

    # Melakukan penggantian kata-kata dalam kolom tertentu
    df_copy[column_name] = df_copy[column_name].apply(
        lambda sentence: ' '.join(replacements.get(word, word) for word in sentence.split())
    )

    return df_copy

In [7]:
#data frame
xx = data.copy()
#kamus
kamus = pd.read_csv('normalisasi.csv')
kamus['original'] = kamus['original'].astype(str)
dictionary = kamus.set_index('original').to_dict()['normalisasi'] #INI YANG SELAMA INI SAYA CARI - CARI WKWK

#ujicoba fungsi
df_resultt = replace_words_in_sentence(xx, 'textDisplay', dictionary)
df_resultt

,textDisplay
0,coba pikir ini jelas salah tetapi sedikit oran...
1,sudah mending pilih anis sudah jelas jelas kan...
2,mba nana harusnnya jangan framing kubu yang ti...
3,saya fanatik jokowi tetapi setelah terjadi rek...
4,assalamualaikum mbak saya koq tidak lihat meng...
...,...
891,kode keras dari najwa kalau gibran sampai maju...
892,saldi isra itu hakim pilihan presiden mereka s...
893,iya dong mubazir kalau tidak dipake
895,setuju


In [8]:
#cek isi komentar apakah masih ada yg kurang baik
print(df_resultt.duplicated().sum())
print(df_resultt.isnull().sum())

8
textDisplay    0
dtype: int64


In [9]:
df_resultt=df_resultt.drop_duplicates()
print(df_resultt.duplicated().sum())
print(df_resultt.isnull().sum())

0
textDisplay    0
dtype: int64


In [10]:
df_resultt

,textDisplay
0,coba pikir ini jelas salah tetapi sedikit oran...
1,sudah mending pilih anis sudah jelas jelas kan...
2,mba nana harusnnya jangan framing kubu yang ti...
3,saya fanatik jokowi tetapi setelah terjadi rek...
4,assalamualaikum mbak saya koq tidak lihat meng...
...,...
890,suara hatiku telah diwakilkan mbak nana
891,kode keras dari najwa kalau gibran sampai maju...
892,saldi isra itu hakim pilihan presiden mereka s...
893,iya dong mubazir kalau tidak dipake


In [ ]:
df_resultt.to_csv('ralatdatasetbersih.csv',index=False)

In [11]:
#TOKENIZE, SATU PERSATU KATA DI PISAH2
from nltk.tokenize import word_tokenize
#NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)
df_resultt['textDisplay'] = df_resultt['textDisplay'].apply(word_tokenize_wrapper)
df_resultt.head()

C:\Users\bowok\AppData\Local\Temp\ipykernel_10556\303265408.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultt['textDisplay'] = df_resultt['textDisplay'].apply(word_tokenize_wrapper)


,textDisplay
0,"[coba, pikir, ini, jelas, salah, tetapi, sedik..."
1,"[sudah, mending, pilih, anis, sudah, jelas, je..."
2,"[mba, nana, harusnnya, jangan, framing, kubu, ..."
3,"[saya, fanatik, jokowi, tetapi, setelah, terja..."
4,"[assalamualaikum, mbak, saya, koq, tidak, liha..."


In [12]:
#STOPWROD
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = pd.read_csv("stopwords.csv")
def stopword_removal(Review):
    filtering = stopwords.words('indonesian','english')
    filtering.extend(sw)
    x = []
    data = []
    def myFunc(x):
        if x in filtering:
            return False
        else:
            return True
    fit = filter(myFunc, Review)
    for x in fit:
        data.append(x)
    return data
df_resultt['textDisplay'] = df_resultt['textDisplay'].apply(stopword_removal)
df_resultt.head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bowok\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\bowok\AppData\Local\Temp\ipykernel_10556\688719749.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultt['textDisplay'] = df_resultt['textDisplay'].apply(stopword_removal)


,textDisplay
0,"[coba, pikir, salah, orang, protes, orang, ora..."
1,"[mending, pilih, anis, inget, kapal, selam, na..."
2,"[mba, nana, harusnnya, framing, kubu, setujuco..."
3,"[fanatik, jokowi, rekayasa, konstitusi, lolosn..."
4,"[assalamualaikum, mbak, koq, lihat, menggebu2n..."
5,"[pelanggaran, moral, etik, berat, dinegara, in..."
6,"[fakta, satunya, orang, indonesia, putusan, ko..."
7,"[setali, uang, paman, ponakan, sama2, malu, pi..."
8,"[berkuasa, anak, indonesia, pintar, ditakuti, ..."
9,"[lawak, negeri, kepala, dinas, pendidikan, kam..."


In [ ]:
df_resultt.to_csv('hasilstopwordbismillah.csv',index=False)

In [ ]:
# proses stemming
!pip install Sastrawi
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(Review):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in Review:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean=[]
    d_clean=" ".join(do)
    print(d_clean)
    return d_clean
df_resultt['textDisplay'] = df_resultt['textDisplay'].apply(stemming)

df_resultt.to_csv('bowiqReviewSentimenbismillah.csv', index=False)
#data_clean = pd.read_csv('bowiqReviewSentimen.csv', encoding='latin1')
#data_clean.head()

In [13]:
import preprocessor as p
from textblob import TextBlob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bowok\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\bowok\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [15]:
sentiment = pd.read_csv('ralatdatasetbersih.csv')
sentiment["textDisplay"] = sentiment["textDisplay"].astype(str) #perlu di konversi ke str agak bisa dibaca str

In [16]:
sentiments = SentimentIntensityAnalyzer()

#tambahkan nilai vader bahasa indonesia scr eksternal
#import nilai bahasa indonesia
lexicon_positif = pd.read_csv('lexicon_positive.csv')
lexicon_negative = pd.read_csv('lexicon_negative.csv')

lexicon_positif['word'] = lexicon_positif['word'].astype(str)
lexicon_negative['word'] = lexicon_negative['word'].astype(str)

dictionary_positif = lexicon_positif.set_index('word').to_dict()['weight'] #INI YANG SELAMA INI SAYA CARI - CARI WKWK
dictionary_negative = lexicon_negative.set_index('word').to_dict()['weight'] #INI YANG SELAMA INI SAYA CARI - CARI WKWK

# Tambahkan kamus lexicon bahasa Indonesia ke dalam VADER
sentiments.lexicon.update(dictionary_positif)
# Tambahkan kamus lexicon bahasa Indonesia ke dalam VADER
sentiments.lexicon.update(dictionary_negative)

In [17]:
sentiment["Positif"] = [sentiments.polarity_scores(i)["pos"]for i in sentiment["textDisplay"]]
sentiment["Negatif"] = [sentiments.polarity_scores(i)["neg"]for i in sentiment["textDisplay"]]
sentiment["Neutral"] = [sentiments.polarity_scores(i)["neu"]for i in sentiment["textDisplay"]]
sentiment['Compound'] = [sentiments.polarity_scores(i)["compound"]for i in sentiment["textDisplay"]]
sentiment.head()

,textDisplay,Positif,Negatif,Neutral,Compound
0,coba pikir ini jelas salah tetapi sedikit oran...,0.000,0.867,0.133,-0.9951
1,sudah mending pilih anis sudah jelas jelas kan...,0.000,0.810,0.190,-0.9882
2,mba nana harusnnya jangan framing kubu yang ti...,0.042,0.803,0.155,-0.9960
3,saya fanatik jokowi tetapi setelah terjadi rek...,0.163,0.488,0.349,-0.9432
4,assalamualaikum mbak saya koq tidak lihat meng...,0.235,0.529,0.235,-0.9517


In [18]:
score = sentiment["Compound"].values
hasil_sentiment =[]
for i in score:
    if  i>= 0.05:
        hasil_sentiment.append('positif')
    elif i <= -0.05:
        hasil_sentiment.append('negatif')
    else:
        hasil_sentiment.append('netral')
        
sentiment["Sentiment"] = hasil_sentiment
sentiment.head()

,textDisplay,Positif,Negatif,Neutral,Compound,Sentiment
0,coba pikir ini jelas salah tetapi sedikit oran...,0.000,0.867,0.133,-0.9951,negatif
1,sudah mending pilih anis sudah jelas jelas kan...,0.000,0.810,0.190,-0.9882,negatif
2,mba nana harusnnya jangan framing kubu yang ti...,0.042,0.803,0.155,-0.9960,negatif
3,saya fanatik jokowi tetapi setelah terjadi rek...,0.163,0.488,0.349,-0.9432,negatif
4,assalamualaikum mbak saya koq tidak lihat meng...,0.235,0.529,0.235,-0.9517,negatif


In [19]:
kolom_drop = ['Positif','Negatif','Neutral','Compound']
sentiment = sentiment.drop(kolom_drop,axis=1)
sentiment.head()

,textDisplay,Sentiment
0,coba pikir ini jelas salah tetapi sedikit oran...,negatif
1,sudah mending pilih anis sudah jelas jelas kan...,negatif
2,mba nana harusnnya jangan framing kubu yang ti...,negatif
3,saya fanatik jokowi tetapi setelah terjadi rek...,negatif
4,assalamualaikum mbak saya koq tidak lihat meng...,negatif


In [20]:
sentiment.to_csv('HasilLabelSementaraPrabowoVader111.csv',index=False)

In [2]:
import pandas as pd
data_clean = pd.read_csv('HasilLabelSementaraPrabowoVader111.csv', encoding='latin1')
data_clean.head()

,textDisplay,Sentiment
0,coba pikir ini jelas salah tetapi sedikit oran...,negatif
1,sudah mending pilih anis sudah jelas jelas kan...,negatif
2,mba nana harusnnya jangan framing kubu yang ti...,negatif
3,saya fanatik jokowi tetapi setelah terjadi rek...,negatif
4,assalamualaikum mbak saya koq tidak lihat meng...,negatif


In [3]:
data_clean = data_clean.astype({'Sentiment' : 'category'})
data_clean = data_clean.astype({'textDisplay' : 'string'})
data_clean.dtypes

textDisplay      string
Sentiment      category
dtype: object

In [4]:
# PROSES TF IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_clean['textDisplay'].astype('U'))

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_clean['textDisplay'].astype('U'))
print(text_tf)

  (0, 3472)	0.29126079287122847
  (0, 4213)	0.20852921055847157
  (0, 1426)	0.2746124630884017
  (0, 368)	0.24615195534251652
  (0, 241)	0.26280028512534337
  (0, 371)	0.17036261897090874
  (0, 244)	0.1476947749639298
  (0, 1908)	0.2736530855776118
  (0, 3766)	0.25363804808718354
  (0, 4791)	0.08074966520315115
  (0, 3294)	0.422506686243543
  (0, 4048)	0.26280028512534337
  (0, 4516)	0.1377927010919622
  (0, 3955)	0.18006870281258644
  (0, 1768)	0.18006870281258644
  (0, 1653)	0.11185199205937243
  (0, 3632)	0.2746124630884017
  (0, 831)	0.2251775403412984
  (1, 2257)	0.24960388057558247
  (1, 934)	0.10655468732998885
  (1, 3943)	0.13307041028326744
  (1, 59)	0.30314006299046076
  (1, 3115)	0.30314006299046076
  (1, 4087)	0.30314006299046076
  (1, 1901)	0.30314006299046076
  :	:
  (882, 3961)	0.34793557078918685
  (882, 479)	0.3139370950249165
  (882, 1474)	0.2689935547287805
  (882, 3935)	0.5483228564347323
  (882, 3640)	0.28648825827417673
  (882, 3471)	0.2799386192606462
  (882, 372

In [5]:
# Import cosine_similarity metrics
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# compute similarity using cosine similarity
cos_sim=cosine_similarity(text_tf, text_tf)

print(cos_sim)

[[1.         0.06749441 0.03773568 ... 0.         0.         0.        ]
 [0.06749441 1.         0.         ... 0.02032444 0.         0.        ]
 [0.03773568 0.         1.         ... 0.         0.0299878  0.        ]
 ...
 [0.         0.02032444 0.         ... 1.         0.         0.        ]
 [0.         0.         0.0299878  ... 0.         1.         0.18173851]
 [0.         0.         0.         ... 0.         0.18173851 1.        ]]


In [9]:
# splitting data 
import collections, numpy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cos_sim, data_clean['Sentiment'], test_size=0.2, random_state=33)
print("Jumlah Data Uji:", X_test.shape)
print("Jumlah Data Latih:",X_train.shape)

pos = (y_test == 'POSITIF').sum()
neg = (y_test == 'NEGATIF').sum()
postrain = (y_train == 'POSITIF').sum()
negtrain = (y_train == 'NEGATIF').sum()
total = pos + neg
print("Jumlah data uji dengan sentimen positif:", pos)
print("Jumlah data uji dengan sentimen negatif:",neg)
print("Jumlah data latih dengan sentimen positif:", postrain)
print("Jumlah data latih dengan sentimen negatif:",negtrain)
data_clean['Sentiment'].value_counts()

Jumlah Data Uji: (177, 885)
Jumlah Data Latih: (708, 885)
Jumlah data uji dengan sentimen positif: 0
Jumlah data uji dengan sentimen negatif: 0
Jumlah data latih dengan sentimen positif: 0
Jumlah data latih dengan sentimen negatif: 0


negatif    764
positif     61
netral      60
Name: Sentiment, dtype: int64

In [13]:
# perform algoritma KNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
predicted = clf.predict(X_test)
print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('===============================================\n')

confusion matrix:
 [[154   5   2]
 [  6   2   0]
 [  7   1   0]]



C:\Users\bowok\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [19]:
tN= confusion_matrix(y_test, predicted)
tN

array([[154,   5,   2],
       [  6,   2,   0],
       [  7,   1,   0]], dtype=int64)